In [25]:
import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)

# import libraries
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import re,random,os
import seaborn as sns
from nltk.corpus import stopwords
import string
from pprint import pprint as pprint

# spacy for basic processing, optional, can use nltk as well(lemmatisation etc.)
import spacy

#gensim for LDA
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt

In [26]:
spacy.cli.download("en_core_web_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 11.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [27]:
df = pd.read_csv('/Users/yokk/Desktop/ARP-Grecale/MaseratiForum/MaseratiForum.csv')

Tokenize each comment (using gensim)
Remove stop words (including punctuations)
Lemmatize (using spacy)

In [28]:
# tokenize using gensims simple_preprocess

def sent_to_words(sentences, deacc=True):  # deacc=True removes punctuations
    for sentence in sentences:
        yield(simple_preprocess(str(sentence)))

# convert to list
data=df['Comment'].values.tolist()
data_words=list(sent_to_words(data))

print(data_words[3])

['thanks', 'for', 'the', 'reply', 'modenacanada', 'so', 'obviously', 'not', 'just', 'right', 'hand', 'drive', 'cars', 'delayed', 'sure', 'they', 'will', 'turn', 'up', 'eventually', 'and', 'yes', 'thinking']


In [29]:
# create a list of stop words
# string.punctuation (from the 'string' module) contains a list of punctuations

from nltk.corpus import stopwords
stop_words= stopwords.words('english') + list(string.punctuation)

In [30]:
# functions for removing stopwords and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def lemmatization(texts,allowed_postags=['NOUN','ADJ','VERB','ADV']):
    """https://spacy.io/api/annotation"""
    texts_out=[]
    for sent in texts:
        doc=nlp(' '.join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [31]:
# call functions
nlp = spacy.load("en_core_web_sm")
# remove stop words
data_words_npstops= remove_stopwords(data_words)

# initialize spacy 'en' model use only tagger since we don;t need parsing or NER
# python3 -m spacey download en
# spacy.cli.download("en")
nlp=spacy.load('en_core_web_sm',disable=['parser', 'ner'])

# lemmatization keeping only noun, adj, vb, adv
data_lemmatized=lemmatization(data_words_npstops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[3])

['thank', 'reply', 'obviously', 'right', 'hand', 'drive', 'car', 'delay', 'sure', 'turn', 'eventually', 'think']


In [32]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 3), (4, 1), (5, 1), (6, 1), (7, 1), (8, 3), (9, 2), (10, 2), (11, 1), (12, 1), (13, 1), (14, 4), (15, 1), (16, 2), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 4), (27, 2), (28, 1), (29, 1), (30, 1), (31, 4), (32, 1), (33, 1), (34, 2), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 2)]]


In [33]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [34]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.162*"assist" + 0.103*"thank" + 0.061*"post" + 0.060*"care" + '
  '0.030*"forum" + 0.027*"provide" + 0.020*"com" + 0.019*"team" + 0.019*"sign" '
  '+ 0.017*"exist"'),
 (1,
  '0.100*"excited" + 0.094*"possible" + 0.030*"note" + 0.023*"appreciate" + '
  '0.021*"exact" + 0.019*"swap" + 0.017*"fact" + 0.016*"suppose" + '
  '0.013*"follow" + 0.011*"white"'),
 (2,
  '0.106*"car" + 0.087*"issue" + 0.047*"also" + 0.040*"software" + 0.033*"get" '
  '+ 0.032*"come" + 0.030*"time" + 0.027*"back" + 0.026*"even" + 0.026*"long"'),
 (3,
  '0.063*"sound" + 0.043*"stop" + 0.036*"car" + 0.031*"sometimes" + '
  '0.026*"message" + 0.024*"ever" + 0.023*"mile" + 0.023*"low" + '
  '0.023*"modena" + 0.022*"also"'),
 (4,
  '0.046*"car" + 0.029*"dealer" + 0.028*"week" + 0.026*"much" + 0.024*"say" + '
  '0.022*"go" + 0.022*"get" + 0.021*"month" + 0.020*"phone" + 0.019*"last"'),
 (5,
  '0.090*"progress" + 0.071*"agree" + 0.053*"production" + 0.050*"quote" + '
  '0.045*"fit" + 0.032*"less" + 0.030*"line" 

You can see the top keywords and weights associated with keywords contributing to topic.
Topics are words with highest probability in topic and the numbers are the probabilities of words appearing in topic distribution.
But looking at keywords can you guess what the topic is?
You may summarize topic-4 as space(In the above figure). Each one may have different topic at particular number , topic 4 might not be in the same place where it is now, it may be in topic 10 or any number.

In [35]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -12.831918666125588

Coherence Score:  0.3130766593343335


In [ ]:
# functions for removing stopwords and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def lemmatization(texts,allowed_postags=['NOUN','ADJ','VERB','ADV']):
    """https://spacy.io/api/annotation"""
    texts_out=[]
    for sent in texts:
        doc=nlp(' '.join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

Lower the perplexity better the model.
Higher the topic coherence, the topic is more human interpretable.


In [38]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(lda_model, corpus, id2word, n_jobs=1)

In [39]:
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.268323  0.071489       1        1  20.566425
2     -0.286023 -0.106166       2        1  16.466042
13    -0.227158 -0.032399       3        1  14.685477
7     -0.118978  0.174077       4        1   7.636355
3     -0.115524 -0.198706       5        1   7.360410
8     -0.060377  0.155467       6        1   4.123931
17     0.013151 -0.082312       7        1   3.788119
11     0.031697 -0.183908       8        1   3.548003
12    -0.067552  0.008828       9        1   3.330629
14    -0.089874  0.106020      10        1   3.230519
16     0.025035 -0.064453      11        1   3.202826
15     0.046409  0.079977      12        1   2.887357
6      0.118383  0.023036      13        1   1.751897
19     0.082894  0.081502      14        1   1.725521
0      0.144561 -0.024149      15        1   1.547577
5      0.124993 -0.009955      16        1   1.278335
10     0.143877  0.046886      17        1   1.215187
1      0.177533 -0.020935      18        1   0.672954
18     0.170358 -0.030762      19        1   0.581512
9      0.154918  0.006462      20        1   0.400924, topic_info=       Term        Freq       Total Category  logprob  loglift
236    work  186.000000  186.000000  Default  30.0000  30.0000
240  system  184.000000  184.000000  Default  29.0000  29.0000
49    drive  226.000000  226.000000  Default  28.0000  28.0000
361    lane  121.000000  121.000000  Default  27.0000  27.0000
7    dealer  341.000000  341.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
63      yet    0.469526   53.582478  Topic20  -5.1956   0.7819
99    forum    0.482662   62.806599  Topic20  -5.1680   0.6507
217    year    0.451238   69.930394  Topic20  -5.2353   0.4759
185   first    0.467257  101.578469  Topic20  -5.2005   0.1374
208    even    0.474462  127.582523  Topic20  -5.1852  -0.0752

[892 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
549      11  0.936472        able
549      18  0.032292        able
593       1  0.864987  absolutely
647       8  0.966972      accept
201       3  0.940743      accord
...     ...       ...         ...
129       3  0.955833      yellow
63        1  0.298605         yet
63        2  0.130640         yet
63        3  0.447908         yet
63        5  0.093314         yet

[1104 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 14, 8, 4, 9, 18, 12, 13, 15, 17, 16, 7, 20, 1, 6, 11, 2, 19, 10])